# MNIST Training Experiment

This notebook contains experiments for the MNIST classification task.

**Warning**: This notebook may not work correctly - some imports are broken.

In [ ]:
# Standard imports
import sys
sys.path.append('../src')

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Import our modules - some might fail
try:
    from train_final_REAL import SimpleCNN, load_data
    print("Imported train_final_REAL successfully")
except ImportError as e:
    print(f"Import failed: {e}")

# Duplicate import - just in case
try:
    from model import SimpleCNN  # This might conflict
except:
    pass

In [ ]:
# Magic numbers copied from train_final_REAL.py
MAGIC_NUMBER_1 = 0.1307
MAGIC_NUMBER_2 = 0.3081
BATCH_SIZE = 64

# These might be wrong
LR = 0.001
EPOCHS = 10

In [ ]:
# Load data - duplicate of train_final_REAL.py load_data
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((MAGIC_NUMBER_1,), (MAGIC_NUMBER_2,))
])

train_dataset = datasets.MNIST('data/raw', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('data/raw', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
# Create model - this duplicates SimpleCNN definition
class SimpleCNNExperiment(nn.Module):
    """Duplicate of SimpleCNN for experiments"""
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = SimpleCNNExperiment()
print(model)

In [ ]:
# Training loop - duplicate of train_final_REAL.py
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

train_losses = []
for epoch in range(3):  # Just 3 epochs for testing
    model.train()
    running_loss = 0
    for i, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if i % 200 == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item():.4f}')
    
    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)
    print(f'Epoch {epoch} complete, Avg Loss: {epoch_loss:.4f}')

In [ ]:
# Plot loss curve
plt.figure(figsize=(10, 6))
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.savefig('../outputs/experiment_loss.png')
plt.show()

In [ ]:
# Evaluation - duplicate logic
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)

accuracy = correct / total
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
# TODO: Add more experiments
# - Different learning rates
# - Different architectures
# - Data augmentation

# This cell is incomplete

In [ ]:
# Old experiments - don't run
if False:
    # This code is deprecated
    pass